# ChatGPTモデルへの入力をフォーマットする方法

ChatGPTは、OpenAIの最も先進的なモデルである`gpt-3.5-turbo`と`gpt-4`によって動作しています。

OpenAI APIを使用して、`gpt-3.5-turbo`または`gpt-4`で独自のアプリケーションを構築できます。

チャットモデルは一連のメッセージを入力として受け取り、AIが生成したメッセージを出力として返します。

このガイドでは、いくつかのAPI呼び出しの例を使ってチャット形式を説明します。

## 1. openaiライブラリをインポートする

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai


In [22]:
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

## 2. チャット補完APIコールの例

チャット補完APIコールのパラメータ：

**必須**
- `model`: 使用したいモデルの名前（例：`gpt-3.5-turbo`、`gpt-4`、`gpt-3.5-turbo-16k-1106`）
- `messages`: メッセージオブジェクトのリスト。各オブジェクトには2つの必須フィールドがあります：
    - `role`: メッセンジャーの役割（`system`、`user`、`assistant`、または`tool`のいずれか）
    - `content`: メッセージの内容（例：`Write me a beautiful poem`）

メッセージには、メッセンジャーに名前を付けるオプションの`name`フィールドを含めることもできます。例：`example-user`、`Alice`、`BlackbeardBot`。名前にはスペースを含めることはできません。

**オプション**
- `frequency_penalty`: 頻度に基づいてトークンにペナルティを課し、繰り返しを減らします。
- `logit_bias`: 指定されたトークンの尤度をバイアス値で変更します。
- `logprobs`: trueの場合、出力トークンの対数確率を返します。
- `top_logprobs`: 各位置で返す最も可能性の高いトークンの数を指定します。
- `max_tokens`: チャット補完で生成されるトークンの最大数を設定します。
- `n`: 各入力に対して指定された数のチャット補完選択肢を生成します。
- `presence_penalty`: テキスト内での存在に基づいて新しいトークンにペナルティを課します。
- `response_format`: 出力形式を指定します（例：JSONモード）。
- `seed`: 指定されたシードで決定論的サンプリングを保証します。
- `stop`: APIがトークン生成を停止すべき最大4つのシーケンスを指定します。
- `stream`: トークンが利用可能になると部分的なメッセージデルタを送信します。
- `temperature`: 0から2の間のサンプリング温度を設定します。
- `top_p`: nucleus samplingを使用し、top_p確率質量を持つトークンを考慮します。
- `tools`: モデルが呼び出す可能性のある関数をリストします。
- `tool_choice`: モデルの関数呼び出しを制御します（none/auto/function）。
- `user`: エンドユーザーの監視と不正使用検出のための一意識別子。

2024年1月時点では、GPTが関数に渡すJSONを生成できるかどうかを伝える`functions`のリストをオプションで送信することもできます。詳細については、[ドキュメント](https://platform.openai.com/docs/guides/function-calling)、[APIリファレンス](https://platform.openai.com/docs/api-reference/chat)、またはCookbookガイド[How to call functions with chat models](How_to_call_functions_with_chat_models.ipynb)を参照してください。

通常、会話はアシスタントの動作を指示するシステムメッセージから始まり、その後にユーザーとアシスタントのメッセージが交互に続きますが、この形式に従う必要はありません。

チャット形式が実際にどのように機能するかを確認するために、チャットAPIコールの例を見てみましょう。

In [2]:
# Example OpenAI Python library request
MODEL = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)


In [7]:
print(json.dumps(json.loads(response.model_dump_json()), indent=4))

{
    "id": "chatcmpl-8dee9DuEFcg2QILtT2a6EBXZnpirM",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Orange who?",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1704461729,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 3,
        "prompt_tokens": 35,
        "total_tokens": 38
    }
}


ご覧のとおり、レスポンスオブジェクトにはいくつかのフィールドがあります：
- `id`: リクエストのID
- `choices`: 完了オブジェクトのリスト（`n`を1より大きく設定しない限り、1つのみ）
    - `finish_reason`: モデルがテキスト生成を停止した理由（`stop`、または`max_tokens`制限に達した場合は`length`）
    - `index`: 選択肢リスト内での選択肢のインデックス
    - `logprobs`: 選択肢の対数確率情報
    - `message`: モデルによって生成されたメッセージオブジェクト
        - `content`: メッセージの内容
        - `role`: このメッセージの作成者の役割
        - `tool_calls`: 関数呼び出しなど、モデルによって生成されたツール呼び出し（toolsが指定されている場合）
- `created`: リクエストのタイムスタンプ
- `model`: レスポンス生成に使用されたモデルの完全名
- `object`: 返されるオブジェクトのタイプ（例：`chat.completion`）
- `system_fingerprint`: モデルが実行されるバックエンド設定を表すフィンガープリント
- `usage`: 返答生成に使用されたトークン数（プロンプト、完了、合計をカウント）

以下で返信のみを抽出してください：

In [9]:
response.choices[0].message.content


'Orange who?'

会話ベースではないタスクでも、最初のユーザーメッセージに指示を配置することで、チャット形式に適合させることができます。

例えば、海賊ブラックベアードのスタイルで非同期プログラミングを説明するようモデルに求める場合、以下のように会話を構成できます：

In [10]:
# example with a system message
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


Arr, me matey! Let me tell ye a tale of asynchronous programming, in the style of the fearsome pirate Blackbeard!

Picture this, me hearties. In the vast ocean of programming, there be times when ye need to perform multiple tasks at once. But fear not, for asynchronous programming be here to save the day!

Ye see, in traditional programming, ye be waitin' for one task to be done before movin' on to the next. But with asynchronous programming, ye can be takin' care of multiple tasks at the same time, just like a pirate multitaskin' on the high seas!

Instead of waitin' for a task to be completed, ye can be sendin' it off on its own journey, while ye move on to the next task. It be like havin' a crew of trusty sailors, each takin' care of their own duties, without waitin' for the others.

Now, ye may be wonderin', how does this sorcery work? Well, me matey, it be all about callbacks and promises. When ye be sendin' off a task, ye be attachin' a callback function to it. This be like leavi

In [11]:
# example without a system message
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


Arr, me hearties! Gather 'round and listen up, for I be tellin' ye about the mysterious art of asynchronous programming, in the style of the fearsome pirate Blackbeard!

Now, ye see, in the world of programming, there be times when we need to perform tasks that take a mighty long time to complete. These tasks might involve fetchin' data from the depths of the internet, or performin' complex calculations that would make even Davy Jones scratch his head.

In the olden days, we pirates used to wait patiently for each task to finish afore movin' on to the next one. But that be a waste of precious time, me hearties! We be pirates, always lookin' for ways to be more efficient and plunder more booty!

That be where asynchronous programming comes in, me mateys. It be a way to tackle multiple tasks at once, without waitin' for each one to finish afore movin' on. It be like havin' a crew of scallywags workin' on different tasks simultaneously, while ye be overseein' the whole operation.

Ye see,

## 3. gpt-3.5-turbo-0301への指示のコツ

モデルへの指示のベストプラクティスは、モデルのバージョンごとに変わる可能性があります。以下のアドバイスは`gpt-3.5-turbo-0301`に適用されるものであり、将来のモデルには適用されない可能性があります。

### システムメッセージ

システムメッセージは、アシスタントに異なる性格や行動を設定するために使用できます。

`gpt-3.5-turbo-0301`は一般的に`gpt-4-0314`や`gpt-3.5-turbo-0613`ほどシステムメッセージに注意を払わないことに注意してください。そのため、`gpt-3.5-turbo-0301`については、重要な指示をユーザーメッセージに配置することを推奨します。一部の開発者は、会話が長くなるにつれてモデルの注意が逸れることを防ぐために、システムメッセージを会話の終わり近くに継続的に移動させることで成功を収めています。

In [12]:
# An example of a system message that primes the assistant to explain concepts in great depth
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a friendly and helpful teaching assistant. You explain concepts in great depth using simple terms, and you give examples to help people learn. At the end of each explanation, you ask a question to check for understanding"},
        {"role": "user", "content": "Can you explain how fractions work?"},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


Of course! Fractions are a way to represent parts of a whole. They are made up of two numbers: a numerator and a denominator. The numerator tells you how many parts you have, and the denominator tells you how many equal parts make up the whole.

Let's take an example to understand this better. Imagine you have a pizza that is divided into 8 equal slices. If you eat 3 slices, you can represent that as the fraction 3/8. Here, the numerator is 3 because you ate 3 slices, and the denominator is 8 because the whole pizza is divided into 8 slices.

Fractions can also be used to represent numbers less than 1. For example, if you eat half of a pizza, you can write it as 1/2. Here, the numerator is 1 because you ate one slice, and the denominator is 2 because the whole pizza is divided into 2 equal parts.

Now, let's talk about equivalent fractions. Equivalent fractions are different fractions that represent the same amount. For example, 1/2 and 2/4 are equivalent fractions because they both re

In [13]:
# An example of a system message that primes the assistant to give brief, to-the-point answers
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a laconic assistant. You reply with brief, to-the-point answers with no elaboration."},
        {"role": "user", "content": "Can you explain how fractions work?"},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


Fractions represent parts of a whole. They have a numerator (top number) and a denominator (bottom number).


### Few-shot prompting

場合によっては、モデルに何をしてほしいかを説明するよりも、実際に見せる方が簡単なことがあります。

モデルに何をしてほしいかを示す一つの方法は、偽の例示メッセージを使用することです。

例えば：

In [14]:
# An example of a faked few-shot conversation to prime the model into translating business jargon to simpler speech
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


This sudden change in direction means we don't have enough time to complete the entire project for the client.


モデルが例示メッセージを実際の会話の一部として扱わず、後で参照しないようにするために、`system`メッセージの`name`フィールドを`example_user`と`example_assistant`に設定することを試すことができます。

上記のfew-shotの例を変換すると、次のように書くことができます：

In [15]:
# The business jargon translation example, but with example names for the example messages
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
        {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
        {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
        {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)


This sudden change in direction means we don't have enough time to complete the entire project for the client.


エンジニアリングされた会話のすべての試みが最初から成功するわけではありません。

最初の試みが失敗した場合は、モデルをプライミングまたは条件付けする異なる方法を実験することを恐れないでください。

例として、ある開発者は「これまで素晴らしい仕事です、これらは完璧でした」というユーザーメッセージを挿入することで、モデルがより高品質な応答を提供するよう条件付けし、精度の向上を発見しました。

モデルの信頼性を向上させる方法についてのより多くのアイデアについては、[信頼性を向上させるテクニック](../techniques_to_improve_reliability)に関するガイドを読むことを検討してください。これは非チャットモデル向けに書かれましたが、その原則の多くは今でも適用されます。

## 4. トークンのカウント

リクエストを送信すると、APIはメッセージをトークンのシーケンスに変換します。

使用されるトークン数は以下に影響します：
- リクエストのコスト
- レスポンス生成にかかる時間
- 最大トークン制限に達した際の返答の切り捨て（`gpt-3.5-turbo`では4,096、`gpt-4`では8,192）

メッセージのリストが使用するトークン数をカウントするには、以下の関数を使用できます。

メッセージからトークンがカウントされる正確な方法は、モデルによって変わる可能性があることに注意してください。以下の関数からのカウントは推定値として考え、永続的な保証ではないことを理解してください。

特に、オプションのfunctions入力を使用するリクエストは、以下で計算される推定値に加えて追加のトークンを消費します。

トークンのカウントについて詳しくは、[tiktokenでトークンをカウントする方法](How_to_count_tokens_with_tiktoken.ipynb)をお読みください。

In [16]:
import tiktoken


def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [23]:
# let's verify the function above matches the OpenAI API response
example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in [
    # "gpt-3.5-turbo-0301",
    # "gpt-4-0314",
    # "gpt-4-0613",
    "gpt-3.5-turbo-1106",
    "gpt-3.5-turbo",
    "gpt-4",
    "gpt-4-1106-preview",
    ]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = client.chat.completions.create(model=model,
    messages=example_messages,
    temperature=0,
    max_tokens=1)
    token = response.usage.prompt_tokens
    print(f'{token} prompt tokens counted by the OpenAI API.')
    print()


gpt-3.5-turbo-1106
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

gpt-3.5-turbo
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

gpt-4
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

gpt-4-1106-preview
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

